# Models comparison: an empiirical comparison between Pooled OLS, Random Effects and Fixed Effects

This is part of a series of projects I want to build in order to have a better understanding of the topics included in the course "Empirical Methods and Impact Analysis" of the University of Bergamo, Italy. For the implementations, I will use python as a programming language, together with some libraries.



In [2]:
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 8.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import statsmodels.api as sm  # for the dataset and Hausman's test
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS
from linearmodels.panel import compare  # to compare the models

In [4]:
# we load the dataset "Grunfeld"
data = sm.datasets.grunfeld.load_pandas()
df = data.data

# we visualize the first rows in order to understand the data
print(df.head())

   invest   value  capital            firm    year
0   317.6  3078.5      2.8  General Motors  1935.0
1   391.8  4661.7     52.6  General Motors  1936.0
2   410.6  5387.1    156.9  General Motors  1937.0
3   257.7  2792.2    209.2  General Motors  1938.0
4   330.8  4313.2    203.4  General Motors  1939.0


In [5]:
df['firm'] = pd.Categorical(df['firm']) #we convert 'firm' in a categorical variable

# we create the index of the panel
# linearmodels will understand that 'firm' is the entity (category)
# and that 'year' is the time (because numeric)
df = df.set_index(['firm', 'year'])

# we add a constant (intercept)
df = sm.add_constant(df, prepend=False)

# we define our Y and our X
Y = df['invest']
X = df[['const', 'value', 'capital']]

print("\nDati pronti per l'analisi (con MultiIndex):")
print(df.head())
print(f"\nTipi di dati dell'indice: {df.index.dtypes}")


Dati pronti per l'analisi (con MultiIndex):
                       invest   value  capital  const
firm           year                                  
General Motors 1935.0   317.6  3078.5      2.8    1.0
               1936.0   391.8  4661.7     52.6    1.0
               1937.0   410.6  5387.1    156.9    1.0
               1938.0   257.7  2792.2    209.2    1.0
               1939.0   330.8  4313.2    203.4    1.0

Tipi di dati dell'indice: firm    category
year     float64
dtype: object


In [6]:
# we define the model pooled OLS
model_pooled = PooledOLS(Y, X)

# we estimate the model
results_pooled = model_pooled.fit()

# we print the results
print("Pooled Model Results:")
print(results_pooled)

Pooled Model Results:
                          PooledOLS Estimation Summary                          
Dep. Variable:                 invest   R-squared:                        0.8179
Estimator:                  PooledOLS   R-squared (Between):              0.8426
No. Observations:                 220   R-squared (Within):               0.7357
Date:                Tue, Nov 11 2025   R-squared (Overall):              0.8179
Time:                        15:03:51   Log-likelihood                   -1301.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      487.28
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,217)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             487.28
      

/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:685: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mu = self._frame.groupby(level=level).mean()
/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:685: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mu = self._frame.groupby(level=level).mean()
/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:645: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and sile

In [7]:
# we define the fixed effects model
# entity_effects=True says to the model to include the fixed effects for 'firm'
model_fe = PanelOLS(Y, X, entity_effects=True)

# we estimate the model
results_fe = model_fe.fit()

# we print the results
print("Fixed Effects Results:")
print(results_fe)

Fixed Effects Results:
                          PanelOLS Estimation Summary                           
Dep. Variable:                 invest   R-squared:                        0.7667
Estimator:                   PanelOLS   R-squared (Between):              0.8193
No. Observations:                 220   R-squared (Within):               0.7667
Date:                Tue, Nov 11 2025   R-squared (Overall):              0.8071
Time:                        15:04:00   Log-likelihood                   -1167.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      340.08
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,207)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             340.08
     

/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:595: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:595: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:685: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [8]:
# we define the random effects model
model_re = RandomEffects(Y, X)

# we estimate the model
results_re = model_re.fit()

# we print the results
print("Random Effects Results:")
print(results_re)

Random Effects Results:
                        RandomEffects Estimation Summary                        
Dep. Variable:                 invest   R-squared:                        0.7700
Estimator:              RandomEffects   R-squared (Between):              0.8204
No. Observations:                 220   R-squared (Within):               0.7666
Date:                Tue, Nov 11 2025   R-squared (Overall):              0.8080
Time:                        15:04:07   Log-likelihood                   -1172.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      363.21
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,217)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             363.21
    

/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:604: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weighted_sum: DataFrame = frame.groupby(level=level).transform("sum")
/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:606: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sum_weights: DataFrame = frame.groupby(level=level).transform("sum")
/usr/local/lib/python3.12/dist-packages/linearmodels/panel/data.py:604: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or o

In [9]:
# visual comparison
# we put all the results in a table
print("Models' Comparison:")
print(compare({'Pooled': results_pooled, 'FE': results_fe, 'RE': results_re},
             stars=True))

Models' Comparison:
                            Model Comparison                           
                                Pooled             FE                RE
-----------------------------------------------------------------------
Dep. Variable                   invest         invest            invest
Estimator                    PooledOLS       PanelOLS     RandomEffects
No. Observations                   220            220               220
Cov. Est.                   Unadjusted     Unadjusted        Unadjusted
R-squared                       0.8179         0.7667            0.7700
R-Squared (Within)              0.7357         0.7667            0.7666
R-Squared (Between)             0.8426         0.8193            0.8204
R-Squared (Overall)             0.8179         0.8071            0.8080
F-statistic                     487.28         340.08            363.21
P-value (F-stat)                0.0000         0.0000            0.0000
=====================     ============   ===

In [10]:
# we load again the original data
data = sm.datasets.grunfeld.load_pandas()
df_orig = data.data

# we calculate the groups' averages (firms' averages)
# we use .transform() to map the firm average on each row
df_orig['value_mean'] = df_orig.groupby('firm')['value'].transform('mean')
df_orig['capital_mean'] = df_orig.groupby('firm')['capital'].transform('mean')

# we set the dataframe for analysis
df_mundlak = df_orig.set_index(['firm', 'year'])
df_mundlak = sm.add_constant(df_mundlak, prepend=False)

# we define Y and X (now X includes averages)
Y_mundlak = df_mundlak['invest']
X_mundlak = df_mundlak[['const', 'value', 'capital', 'value_mean', 'capital_mean']]

# we estimate the auxiliary RE model (with robust errors)
model_mundlak_re = RandomEffects(Y_mundlak, X_mundlak)
results_mundlak_re = model_mundlak_re.fit(cov_type='robust')

print("Robust Test (Mundlak-Woolridge)")
print(results_mundlak_re)# --- CODICE PRECEDENTE (lascialo invariato) ---
print("Wald Test on averages")
variabili_media = ['value_mean', 'capital_mean']
wald_test = results_mundlak_re.wald_test(formula='value_mean = 0, capital_mean = 0')
print(wald_test)


# we use try-except to try for both the pval and p_value functions
try:
    p_value_estratto = wald_test.pval
except AttributeError:
    try:
        p_value_estratto = wald_test.p_value
    except AttributeError:
        print("\nERROR")
        print("look at the test's output and manually insert the p-value.")
        p_value_estratto = 1.0 # default value to avoid errors

if p_value_estratto < 0.05:
    print(f"\nResult: P-value ({p_value_estratto:.4f}) is low (< 0.05). Reject H0.")
    print("==> Not observable effects are correlated with regressors")
    print("==> Choose FIXED EFFECTS.")
else:
    print(f"\nResutl: P-value ({p_value_estratto:.4f}) is high (>= 0.05). Not rejecting H0.")
    print("==> RE model is valid. Choose RE.")

Robust Test (Mundlak-Woolridge)
                        RandomEffects Estimation Summary                        
Dep. Variable:                 invest   R-squared:                        0.7714
Estimator:              RandomEffects   R-squared (Between):              0.8644
No. Observations:                 220   R-squared (Within):               0.7667
Date:                Tue, Nov 11 2025   R-squared (Overall):              0.8418
Time:                        15:04:28   Log-likelihood                   -1170.6
Cov. Estimator:                Robust                                           
                                        F-statistic:                      181.33
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(4,215)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             21.